In [105]:
import pandas as pd
import geopandas as gpd
import pickle
import igraph
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmasher as cmr
import colorsys
import shapely
import geopandas
import importlib
from adjustText import adjust_text
import matplotlib.patheffects as PathEffects
from mpmath import mp # pip install mpmath
from scipy import integrate
import matplotlib
import xnetwork as xnet
import numpy as np
from tqdm.auto import tqdm

mp.dps = 300
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

%matplotlib inline
%config InlineBackend.figure_format='retina'


In [98]:

# source: https://github.com/aekpalakorn/python-backbone-network/blob/master/backbone.py
def disparity_filter(g):
	total_vtx = g.vcount()
	g.es['alpha_ij'] = 1

	for v in range(total_vtx):
		edges = g.incident(v)

		k = len(edges)
		if k > 1:
			sum_w = mp.mpf(sum([g.es[e]['weight'] for e in edges]))
			for e in edges:
				w = g.es[e]['weight']
				p_ij = mp.mpf(w)/sum_w
				alpha_ij = 1 - (k-1) * integrate.quad(lambda x: (1-x)**(k-2), 0, p_ij)[0]
				g.es[e]['alpha_ij'] = min(alpha_ij,g.es[e]['alpha_ij'])

def alpha_cut(alpha,g):
	g_copy = g.copy()
	to_delete = g_copy.es.select(alpha_ij_ge=alpha)
	g_copy.delete_edges(to_delete)
	return g_copy

def get_largest_component_size(g):
	components = g.components()
	giant = components.giant()
	return giant.vcount()

def get_best_cut(net,preserve_percent,a_min,a_max):
		a_min = mp.mpf(a_min)
		a_max = mp.mpf(a_max)

		error = 0.015
		largest_size = get_largest_component_size(net)

		min_erro = 1000
		a_min_erro = 0.0

		def get_current_percent(a):
			nonlocal min_erro, a_min_erro, a_min, a_max
			cuted_net = alpha_cut(a,net)
		# print('number of edges',cuted_net.ecount())

			preserved_size = get_largest_component_size(cuted_net)
		# print('preserved size',preserved_size)

			current_percent = mp.mpf(preserved_size)/mp.mpf(largest_size)

			if min_erro > abs(current_percent-preserve_percent):
				min_erro = abs(current_percent-preserve_percent)
				a_min_erro = a

			return cuted_net,current_percent,a

		i = 0

		a_min_perc = mp.mpf(get_largest_component_size(alpha_cut(a_min,net)))/mp.mpf(largest_size)
		a_max_perc = mp.mpf(get_largest_component_size(alpha_cut(a_max,net)))/mp.mpf(largest_size)

		a = 0.0

		while True:
			if i > 100:
				cuted_net = alpha_cut(a_min_erro,net)

				print('error infinity loop')
				print('alpha %.2f; preserved %.2f' % (a_min_erro,min_erro+preserve_percent))
				print()
				return cuted_net

			i += 1
				
			a = (a_min+a_max)/2

			cuted_net,current_percent,a = get_current_percent(a)
			current_erro = current_percent-preserve_percent
			
			if abs(current_erro) < error:
				print('total iterations to find the graph',i)
				print('alpha %.2f; preserved %.2f' % (a,current_percent))
				print()

				return cuted_net

			if (a_min_perc-preserve_percent)*(current_percent-preserve_percent) > 0:
				a_min = a
				a_min_perc = current_percent
			else:
				a_max = a
				a_max_perc = current_percent

def apply_backbone(net,a_min,a_max,preserve=0.8):
	disparity_filter(net)
	best = get_best_cut(net,preserve,a_min,a_max)
	return best

In [127]:
#Generate map code

desiredEdgesCount = 5000

projectsStartYear = {
  "ATLAS":1990,
  "IceCube":2000,
  "LIGO":1990,
  "BaBar":1990,
}

currentYear = 2020;
minWeight = 1 
for projectName,startYear in tqdm(projectsStartYear.items()):
  g = xnet.xnet2igraph("../Data/Networks/network_%s_%d_%d_geo.xnet"%(projectName,startYear,currentYear));
  g.vs["Strength"] = g.strength(weights="weight")
  nodeYears = [set() for _ in range(g.vcount())];
  for edge in g.es:
    edgeYears = {int(year) for year in edge["years"].split(" ")}
    nodeYears[edge.source].update(edgeYears)
    nodeYears[edge.target].update(edgeYears)
  g.vs["years"] = [" ".join(["%d"%year for year in years]) for years in nodeYears]
  
#   edgesRemoved = [i for i,w in enumerate(g.es["weight"]) if w>minWeight]
#   g.delete_edges(edgesRemoved)
  
  backbone = g.copy()
  disparity_filter(backbone)
  topPercentile = desiredEdgesCount/g.ecount()*100
#   backbone.es["weight"] = 1.0 - np.array(backbone.es["alpha_ij"])
  
  # backboneGiant = backbone.components().giant();
  
  xnet.igraph2xnet(backbone,"../Data/ToInstitutionsMap/institutions_%s.xnet"%(projectName));
  pthreshold = np.percentile(np.array(backbone.es["alpha_ij"]),topPercentile)
  edgesRemoved = [i for i,w in enumerate(backbone.es["alpha_ij"]) if w>pthreshold]
  backbone.delete_edges(edgesRemoved)
  
  ratio = backbone.ecount()/g.ecount()
  print("Edges: %d (%.0f %%)"%(backbone.ecount(),ratio*100))
  xnet.igraph2xnet(backbone,"../Data/ToInstitutionsMap/institutions_backbone_%s.xnet"%(projectName));
  
#   edgesRemoved = [i for i,w in enumerate(backbone.es["weight"]) if w<0.95]
#   pthreshold = -np.percentile(-np.array(backbone.es["weight"]),topPercentile)
#   backbone.delete_edges(edgesRemoved)
#   backboneGiant = backbone.components().giant();

#   ratio = backbone.ecount()/g.ecount()
#   print("Edges: %d (%.0f %%)"%(backbone.ecount(),ratio*100))
  
  

  0%|          | 0/4 [00:00<?, ?it/s]

Edges: 5000 (3 %)
Edges: 5029 (3 %)
Edges: 5000 (2 %)
Edges: 5000 (15 %)


In [ ]:
#select the project

desiredEdgesCount = 5000

# projectName = "ATLAS"
# startYear = 1990
# currentYear = 2007

# projectName = "IceCube"
# startYear = 2000

# projectName = "LIGO"
# startYear = 1990

projectName = "BaBar"
startYear = 1990
currentYear = 2020


g = xnet.xnet2igraph("../Data/Networks/network_%s_%d_%d_geo.xnet"%(projectName,startYear,currentYear))
g.simplify(combine_edges={"weight":"sum","year":"min"});

topPercentile = desiredEdgesCount/g.ecount()*100

positions = np.array(g.vs["Position"])
g.vs["lng"] = [a for a,b in positions]
g.vs["lat"] = [b for a,b in positions]
strengths = g.strength(weights="weight")
ranked = sorted(list(range(g.vcount())),key=lambda i:strengths[i],reverse=True)
backbone = g.copy()
disparity_filter(backbone)
print("converting")
backbone.es["weight"] = 1.0-np.array(backbone.es["alpha_ij"])

pthreshold = -np.percentile(-np.array(backbone.es["weight"]),topPercentile)
edgesRemoved = [i for i,w in enumerate(backbone.es["weight"]) if w<0.95]
backbone.delete_edges(edgesRemoved)
# backboneGiant = backbone.components().giant();

ratio = backbone.ecount()/g.ecount()
print("Edges: %d (%.0f %%)"%(backbone.ecount(),ratio*100))

In [5]:
edgesValues = []
weights = []
years = []
for edge in backbone.es():
  weigth = edge["weight"]
  sourcePosition = positions[edge.source];
  targetPosition = positions[edge.target];
  edgesValues.append([sourcePosition[0],sourcePosition[1],targetPosition[0],targetPosition[1]]);
  weights.append(weigth)
  years.append(edge["year"])
edgesValues = np.array(edgesValues)
years = np.array(years)
len(edgesValues)

In [ ]:
import ForcedirectedEdgeBundling as feb
import importlib
#importlib.reload(feb)
# edgesValues = np.array([[0,0,20,80],[0,80,20,0]])
edges,indices = feb.array2edges(edgesValues*100)
# Check size (small edges are removed)
len(edges)
import usageHelper

# Plot raw for comparison
input_lines = feb.edges2lines(edges)
# plot_lines_on_map(input_lines, footer='Raw commuting trips')

# Plot trips after Force-directed Edge Bungling
# usageHelper.plot_lines_on_map(output_lines, footer='Commuting trips processed with Force-directed Edge Bungling')

# lowe compatibility threshold
feb.K = 0.001
feb.S_initial = 0.1
feb.compatibility_threshold = 0.2
feb.P_initial=1
feb.PARALLEL = False
output_lines = feb.forcebundle(edges,List(np.array(weights,dtype=np.float32)[indices]))
input_lines = output_lines
points_lines = [[(point.x,point.y) for point in edge] for edge in input_lines]

In [ ]:
if(True):
  with open("%s_%d_data.p"%(projectName,currentYear), "wb" ) as fd:
    pickle.dump(points_lines,fd)

In [95]:
with open( "%s_%d_data.p"%(projectName,currentYear), "rb" ) as fd:
  points_lines = pickle.load(fd)

In [ ]:

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    '''
    https://stackoverflow.com/a/18926541
    '''
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:

def scale_lightness(rgb, scale_l):
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    return colorsys.hls_to_rgb(h, min(1, l * scale_l), s = s)


In [140]:
import geopandas as gpd
import matplotlib
from shapely.geometry import LineString, Point
import contextily as ctx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from shapely.geometry import Polygon
from collections import Counter


displayLabelIndices = ranked[0:15]

weights = np.array(backbone.es["weight"])[indices]
years = np.array(backbone.es["year"])[indices].astype(dtype=np.int)
sources = np.array([e.source for e in backbone.es])[indices]
targets = np.array([e.target for e in backbone.es])[indices]

institutionColors = {
  "research center": matplotlib.colors.to_rgb("#1f77b4"),
  "university": matplotlib.colors.to_rgb("#ff7f0e"),
  "college": matplotlib.colors.to_rgb("#2ca02c"),
  "other":matplotlib.colors.to_rgb("#d62728")
}

lines = [];
footer=None;
zoom=4
plotInsight = False; # Change to true to generate the insight
# insightBox = (-16.6744469318,32.3897103583,46.8737194139,67.1850344077)
insightBoxes = {
  "Europe" : (-13.1223348937,35.7612855453,38.2681168633,59.2471716978),
  "US" : (-128.7694367362,22.687516566,-59.0938862754,53.1430883873)
}
insightName = "US" #Change to Europe to see Europe map
insightBox = insightBoxes[insightName]
figsize=(18, 18);
  
if(plotInsight):
  figsize=(figsize[0]/2.5, figsize[1]/2.5);
  
save_filename="../Figures/%s_%s_map_%d.pdf"%(projectName,"_%s"%insightName if plotInsight else "",currentYear);
for edge_idx, edge in enumerate(points_lines):
    points = []

    for point in edge:
        points.append(Point(point[0]/100, point[1]/100))

    lines.append(LineString(points))

points = []
for pos in positions:
  points.append(Point(pos[0], pos[1]))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

insightPolygon = Polygon([(insightBox[0], insightBox[1]), (insightBox[0], insightBox[3]), (insightBox[2], insightBox[3]), (insightBox[2], insightBox[1]), (insightBox[0], insightBox[1])])
poly_gdf = gpd.GeoDataFrame([1], geometry=[insightPolygon], crs=world.crs)

glinesSeries = gpd.GeoSeries(lines)
gdfSeries = gpd.GeoSeries(points)
glinesSeries.crs = {'init': 'epsg:5361'}

gdfLines = gpd.GeoDataFrame(geometry = glinesSeries)

gdfPoints = gpd.GeoDataFrame(geometry=gdfSeries)

gpd.GeoDataFrame(geometry = glinesSeries)


# gdf["years"] = years
# gdf["weight"] = weights
# gdf = gdf.to_crs({'init': 'epsg:3857'})
colormap = truncate_colormap(plt.cm.viridis, 0, 0.95)
colormap = cmr.eclipse
 #or any other colormap
normalizeYears = matplotlib.colors.Normalize(vmin=np.min(years), vmax=np.max(years))
normalizeWeights = matplotlib.colors.Normalize(vmin=np.min(weights), vmax=np.max(weights))
# plt.scatter(x, y, c=z, s=5, cmap=colormap, norm=normalize, marker='*')
opacities = [normalizeWeights(weight)*0.1+0.1 for weight in weights]
lwidths = [normalizeWeights(weight)*1+0.25 for weight in weights]
yearColors = [tuple(colormap(normalizeYears(year))[0:3])+(opacities[index],) for index,year in enumerate(years)]
fieldColors = [institutionColors[field] if field in institutionColors else institutionColors["other"] for field in backbone.vs["Field"]]

labelsData = [(positions[i][0],positions[i][1],backbone.vs["Name"][i],scale_lightness(fieldColors[i],0.75)) for i in displayLabelIndices]

gdfPoints["ranks"] = ranked
gdfPoints["color"] = fieldColors
gdfPoints["darkColor"] = [scale_lightness(color,0.75) for color in fieldColors]
gdfPoints["label"] = list(backbone.vs["Name"])
gdfPoints["lng"] = backbone.vs["lng"]
gdfPoints["lat"] = backbone.vs["lat"]

gdfPoints["lng"] = backbone.vs["lng"]
gdfPoints["lat"] = backbone.vs["lat"]

gdfLines["width"] = lwidths
gdfLines["color"] = yearColors
gdfLines["source"] = sources
gdfLines["target"] = targets


gdfPointsTop = gdfPoints.iloc[displayLabelIndices]

if plotInsight:
  gdfPoints = gpd.clip(gdfPoints, insightPolygon)
  gdfPointsTop = gpd.clip(gdfPointsTop, insightPolygon)
  gdfLines = gpd.clip(gdfLines,insightPolygon)

validIndices = set(gdfPoints.index)
gdfLines = gdfLines[gdfLines['source'].isin(validIndices)&gdfLines['target'].isin(validIndices)]
# fig = plt.figure(figsize=figsize)

# ax = fig.add_subplot(111)
fig, ax = plt.subplots(1, 1,figsize=figsize)
# if(plotInsight):
#   ax.axis(insightBox)

if(plotInsight):
  world = gpd.clip(world, insightPolygon)

world.plot(ax=ax,color='#bbbbbb')

poly_gdf.boundary.plot(ax=ax, color="#666666",linestyle="dashed",alpha=0.75,zorder=4)

gdfLines.plot(linewidth=gdfLines["width"], color=gdfLines["color"],ax=ax)
gdfPoints.plot(linewidth=0.25,ax=ax,edgecolors="#444444",facecolors=gdfPoints["color"], markersize=20,
               color='#222222', alpha=1.0,zorder=5)

xmin, xmax, ymin, ymax = ax.axis()
ax.axis((xmin, xmax, ymin, ymax))

# ctx.add_basemap(ax)
#ax.set_xlim(-7885000, -7840000)
# ax.set_ylim(-33.65, -33.3)
plt.axis('off')
if footer:
    ax.set_title(footer)
#plt.annotate('texto', (0, 0), (.5, .01), fontsize=14, textcoords='axes fraction')

gdfPointsTop.plot(linewidth=0.5,
                                    ax=ax,edgecolors="#444444",
                                    facecolors=gdfPointsTop["darkColor"],
                                    markersize=50,
                                    color='#111111', alpha=1.0,zorder=6)

labelsData = zip(gdfPointsTop["lng"],gdfPointsTop["lat"],gdfPointsTop["label"],gdfPointsTop["darkColor"])
texts = [ax.text(lng, lat, label,c=labelColor,fontsize=9, ha='center', va='center', zorder=10) for lng,lat,label,labelColor in labelsData]

adjust_text(texts, arrowprops=dict(arrowstyle='wedge', color=(0.1,0.1,0.1,0.7)),zorder=9)

for text in texts:
  text.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
  
if(not plotInsight):
  cbaxes = fig.add_axes([0.17, 0.4, 1/6, 0.01])
  cb1 = matplotlib.colorbar.ColorbarBase(cbaxes, cmap=colormap,
                                  norm=normalizeYears,
                                  label="Year of first collaboration",
                                  format="%d",
                                  orientation='horizontal')

  legend_elements = [Line2D([0], [0], marker='o', color="#FFFFFF", label=label,linewidth=0.25,
                            markerfacecolor=color, markersize=10) for label,color in institutionColors.items()]

  # Create the figure
  ax.legend(handles=legend_elements,loc="upper right",bbox_to_anchor=(0.17,0.42),frameon=False)

ax.set_axis_off()
ax.set_axis_off()

if save_filename:
    plt.savefig(save_filename, bbox_inches='tight')
#     plt.savefig(save_filename)
plt.show()
plt.close()
